In [3]:
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.cross_validation import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV

y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0


def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall==0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
seq=[]
m6a_2614_sequence="/home02/chenhuangrong/RFH_reverse_feature.csv"
RNA_code='ACGU'
k=1
interval=1
divided_num=10.0
division_num=10
seq=pd.read_csv(m6a_2614_sequence,header=None,index_col=None)
seq=seq.values
positive_seq=seq[0:1307,:]
# X_train, X_test, y_train, y_test = cross_validation.train_test_split(train_data, train_target, test_size=0.1, random_state=0)
negative_seq=seq[1307:2614,:]
kf = KFold(n_splits=division_num,shuffle=False)  
for train_index , test_index in kf.split(positive_seq):  
    positive_df=pd.DataFrame(positive_seq)
    positive_x_train=positive_df.iloc[train_index,:]
    positive_y_train=positive_df.iloc[test_index,:]
    negative_df=pd.DataFrame(negative_seq)
    negative_x_train=negative_df.iloc[train_index,:]
    negative_y_train=negative_df.iloc[test_index,:]
    positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
    positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
    
    X_train = np.array(positive_negative_x_train)
    Y_train = list(map(lambda x: 1, xrange(len(positive_negative_x_train) / 2)))
    Y2_train = list(map(lambda x: 0, xrange(len(positive_negative_x_train) / 2)))
    Y_train.extend(Y2_train)
    Y_train = np.array(Y_train)
    
    X_test = np.array(positive_negative_y_train)
    Y_test  = list(map(lambda x: 1, xrange(len(positive_negative_y_train) / 2)))
    Y2_test  = list(map(lambda x: 0, xrange(len(positive_negative_y_train) / 2)))
    Y_test.extend(Y2_test )
    Y_test  = np.array(Y_test)
    print "train and test samples finished"
    svc = svm.SVC(probability=True)
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,28)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,28))}
    print "begin grid search"
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X_train, Y_train)
    C=clf.best_params_['C']
    y_pred_prob=clf.predict_proba(X_test)
    gamma=clf.best_params_['gamma']
    print 'c:',C,'gamma:',gamma
    y_pred=clf.predict(X_test)
    
    y_pred_prob_all.extend(y_pred_prob)
    y_pred_all.extend(y_pred)
    Y_all.extend(Y_test)
#     ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
    ACC=metrics.accuracy_score(Y_test,y_pred)
    print ACC
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
    F1_Score=metrics.f1_score(Y_test, y_pred)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y_test, y_pred)
    
    pos=TP+FN
    neg=FP+TN
    ACC_all=ACC_all+ACC
    print "ACC_all:",ACC_all
    precision_all=precision_all+precision
    recall_all=recall_all+recall
    SN_all=SN_all+SN
    SP_all=SP_all+SP
    GM_all=GM_all+GM
    TP_all=TP_all+TP
    TN_all=TN_all+TN
    FP_all=FP_all+FP
    FN_all=FN_all+FN
    F_measure_all=F_measure_all+F_measure
    F1_Score_all=F1_Score_all+F1_Score
    pos_all=pos_all+pos
    neg_all=neg_all+neg
    MCC_all=MCC_all+MCC
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
pd.DataFrame(np.matrix(all_y).T).to_csv('/home02/chenhuangrong/RFH_revised_.csv',header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all,pos_all,neg_all]]]
print savedata
easy_excel.save("svm_independent_test",[str(X_train.shape[1])],savedata,'/home02/chenhuangrong/RFH_revised_.xls')

train and test samples finished
begin grid search
c: 1.25992104989 gamma: 0.0374020008171
0.748091603053
ACC_all: 0.748091603053
train and test samples finished
begin grid search
c: 1.05268520264 gamma: 0.0767492411507
0.751908396947
ACC_all: 1.5
train and test samples finished
begin grid search
c: 0.879536170895 gamma: 0.0535777489283
0.801526717557
ACC_all: 2.30152671756
train and test samples finished
begin grid search
c: 0.879536170895 gamma: 0.0447651092838
0.725190839695
ACC_all: 3.02671755725
train and test samples finished
begin grid search
c: 0.513002242354 gamma: 0.0447651092838
0.69465648855
ACC_all: 3.7213740458
train and test samples finished
begin grid search
c: 0.734867246138 gamma: 0.03125
0.721374045802
ACC_all: 4.4427480916
train and test samples finished
begin grid search
c: 0.428621991427 gamma: 0.03125
0.748091603053
ACC_all: 5.19083969466
train and test samples finished
begin grid search
c: 0.879536170895 gamma: 0.0918584057672
0.707692307692
ACC_all: 5.8985320023

True